# Estudio de los datos de Ventas de una Ferretería

Autor: Diana Chacón Ocariz

## Contexto:

Se trata de una pequeña ferretería que maneja un poco más de 3.000 productos distintos. Poseen un software de gestión genérico que les provee una gran cantidad de reportes, básicamente tablas con números, díficiles de analizar (un reporte puede constar de varias decenas de páginas).


## Objetivos del negocio:

**Tener más visibilidad sobre las ventas para poder mejorar el proceso de compras y la toma de decisiones en general:** 

    - Poder analizar objetivamente las ventas
    - Determinar los productos que podrían entrar en rotura de stock al final de un período
    - Identificar los productos menos vendidos
    - Identificar patrones en el comportamiento de las ventas
    

## Objetivos académicos:

    - Estudiar un caso real, con datos reales y cuyo resultado pueda ayudar a alguien a resolver un problema. 
    - Demostrar que la ciencia de datos también puede ayudar a las PYMES
    - Conocer y practicar el uso de herramientas de ciencia de datos
    
## Fuentes de datos:

Los datos provienen de reportes sacados del software de gestión de la empresa. Se trata de archivos .xls que contienen sólo los datos de reportes sobre ventas por producto (2021 y 2022). 

In [1]:
# Librerías utilizadas

import os
import glob
from pathlib import Path

import datetime 

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import altair as alt

%matplotlib inline

## Notebook 1: Carga y Limpieza de Datos:

Una vez leídos los datos y luego de una primera limpieza de los DF, se guardarán en archivos **.parquet** que serán utilizados más tarde en el EDA.

También se guardarán en archivos **.xlsx** que se utilizarán en la creación de reportes en Google Data Studio para failicitar el análisis por parte de los gerentes de la ferretería.

In [2]:
BASE_DIR = Path.cwd()
BASE_DIR

PosixPath('/home/diana/Documentos/Ciencia de Datos/Proyecto Ventas')

In [3]:
# Lectura de los archivos y creación de un DF con todos los datos
    
def read_files(FILES, TYPE_FILE=True):

    df = pd.DataFrame()

    for filename in glob.glob(f"{BASE_DIR / FILES}"):
        df_aux = pd.read_excel(filename)
        
        if TYPE_FILE:
            type_file = Path(filename).name[4:6]
            df_aux['Tipo'] = type_file

        df = pd.concat([df, df_aux])

    return df

## Carga de Datos

In [4]:
FILES_VENTAS = 'datos/in/art*.xls'
FILES_VENTAS

'datos/in/art*.xls'

In [5]:
[Path(filename).name for filename in glob.glob(f"{BASE_DIR / FILES_VENTAS}")]

['art_fa_2022.xls', 'art_ne_2021.xls', 'art_ne_2022.xls', 'art_fa_2021.xls']

In [6]:
%%time

df = read_files(FILES_VENTAS)

df

CPU times: user 2.07 s, sys: 59.6 ms, total: 2.13 s
Wall time: 2.17 s


,Número,Reng,Emisión,Cliente,Vendedor,Almacén,Cantidad,Unid.,Precio Unitario,Monto Base,I.V.A.,Otros,Neto,Tipo
0,00001,NaN,PEGA DE CONTACTO/PEGA ZAPATERA (90 ML) ENVASADO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
1,0000006366,1,2022-01-03 09:00:00,304459.0,1.00,1.00,1.0,UNI,8.64,8.64,1.38,0.0,10.02,fa
2,0000006388,1,2022-01-07 11:40:00,18419125.0,6.00,1.00,1.0,UNI,8.91,8.91,1.43,0.0,10.34,fa
3,Sub-Totales:,2,17.55,2.81,0.00,20.36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
4,00005,NaN,"NIPLE PLASTICO 1"" * 13CMS",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17659,0000004458,15,2021-05-15 11:10:00,15862673,7.00,1.00,1.0,UNI,10.00,10.00,1.60,0.0,11.60,fa
17660,0000005244,2,2021-06-15 10:32:00,15927422,1.00,1.00,1.0,UNI,4.55,4.55,0.00,0.0,4.55,fa
17661,0000005711,3,2021-07-14 10:52:00,13306742,14.00,1.00,1.0,UNI,330.00,330.00,52.80,0.0,382.80,fa
17662,Sub-Totales:,4,366.47,57.91,0.00,424.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa


In [7]:
df.describe()

,Vendedor,Almacén,Cantidad,Precio Unitario,Monto Base,I.V.A.,Otros,Neto
count,33640.000000,33640.000000,28925.000000,28925.000000,28925.000000,28925.000000,28925.000000,28925.000000
mean,8.006907,81.789050,4.566491,33.855537,46.368078,0.692567,0.000006,47.060651
std,4.591927,4574.323653,21.901486,1338.349090,1505.952259,7.324589,0.000764,1506.173378
min,0.000000,0.010000,0.020000,0.010000,0.010000,0.000000,0.000000,0.010000
25%,7.000000,1.000000,1.000000,1.980000,2.600000,0.000000,0.000000,2.700000
50%,10.000000,1.000000,1.000000,5.560000,7.500000,0.000000,0.000000,7.660000
75%,11.000000,1.000000,3.000000,14.690000,20.500000,0.000000,0.000000,21.000000
max,14.000000,618970.850000,2000.000000,212610.600460,212610.600000,846.000000,0.120000,212610.600000


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38351 entries, 0 to 17663
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Número            38351 non-null  object 
 1   Reng              34207 non-null  object 
 2   Emisión           38351 non-null  object 
 3   Cliente           33640 non-null  object 
 4   Vendedor          33640 non-null  float64
 5   Almacén           33640 non-null  float64
 6   Cantidad          28925 non-null  float64
 7   Unid.             28925 non-null  object 
 8   Precio Unitario   28925 non-null  float64
 9   Monto Base        28925 non-null  float64
 10  I.V.A.            28925 non-null  float64
 11  Otros             28925 non-null  float64
 12  Neto              28925 non-null  float64
 13  Tipo              38351 non-null  object 
dtypes: float64(8), object(6)
memory usage: 4.4+ MB


## Limpieza y transformación de los datos:

Aunque los datos no tienen ningún formato especial de Excel, tienen la forma de un reporte con totales, subtotales y datos agrupados por producto

- **Eliminación de totales:** Eliminamos las lineas que continen "total" ya que son los totales y subtotales de los reportes
- **Eliminación de información no relevante:** Conservaremos solo las siguientes columnas: Número, Emisión, Cliente, Vendedor, Cantidad, Neto y Tipo 
- **Construcción del DF definitvo:** Recorreremos el DF para recuperar la información por producto y crearemos un nuevo DF con los datos definitivos
- **Cambio tipos columnas:** Cambiamos el tipo a la columna Vendedor para que sea de tipo entero. Transformamos el campo de fecha para que sea de tipo datetime

### Eliminación de totales y subtotales

In [9]:
# Buscamos las filas de totales y subtotales para eliminarlas
df[df.Número.str.contains('Totales')]


,Número,Reng,Emisión,Cliente,Vendedor,Almacén,Cantidad,Unid.,Precio Unitario,Monto Base,I.V.A.,Otros,Neto,Tipo
3,Sub-Totales:,2,17.55,2.81,0.00,20.36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
7,Sub-Totales:,25,133.08,21.29,0.00,154.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
11,Sub-Totales:,2,53.11,0.0,0.00,53.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
14,Sub-Totales:,2,17.74,0.0,0.00,17.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
19,Sub-Totales:,3,9.64,1.23,0.00,10.87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17630,Sub-Totales:,1,28.5,4.56,0.00,33.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
17633,Sub-Totales:,1,6.25,0,0.00,6.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
17656,Sub-Totales:,86.68,637.51,0,0.00,637.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
17662,Sub-Totales:,4,366.47,57.91,0.00,424.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa


In [10]:
tot_idx = df[df.Número.str.contains('Totales')].index
#tot_idx
df_aux = df.drop(tot_idx)
df_aux

,Número,Reng,Emisión,Cliente,Vendedor,Almacén,Cantidad,Unid.,Precio Unitario,Monto Base,I.V.A.,Otros,Neto,Tipo
0,00001,NaN,PEGA DE CONTACTO/PEGA ZAPATERA (90 ML) ENVASADO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
1,0000006366,1,2022-01-03 09:00:00,304459.0,1.0,1.0,1.0,UNI,8.64,8.64,1.38,0.0,10.02,fa
2,0000006388,1,2022-01-07 11:40:00,18419125.0,6.0,1.0,1.0,UNI,8.91,8.91,1.43,0.0,10.34,fa
4,00005,NaN,"NIPLE PLASTICO 1"" * 13CMS",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
6,0000006409,1,2022-01-12 10:51:00,13763194.0,7.0,1.0,5.0,UNI,29.40,29.40,4.70,0.0,34.10,fa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17657,GEN2,CON IVA,GENERICO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
17658,0000003948,1,2021-04-26 11:19:00,13763881,7.0,1.0,1.0,UNI,21.92,21.92,3.51,0.0,25.43,fa
17659,0000004458,15,2021-05-15 11:10:00,15862673,7.0,1.0,1.0,UNI,10.00,10.00,1.60,0.0,11.60,fa
17660,0000005244,2,2021-06-15 10:32:00,15927422,1.0,1.0,1.0,UNI,4.55,4.55,0.00,0.0,4.55,fa


### Eliminación de columnas no relevantes

In [11]:
# Cambiamos el nombre de las columnas
df_aux.columns

Index(['Número', 'Reng', 'Emisión ', 'Cliente ', 'Vendedor', 'Almacén',
       'Cantidad', 'Unid.', 'Precio Unitario ', 'Monto Base', 'I.V.A.',
       'Otros', 'Neto', 'Tipo'],
      dtype='object')

In [12]:
cols = ['num', 'reng', 'fecha', 'cliente', 'vendedor', 'almacen', 'cantidad',
       'und', 'precio', 'base', 'iva', 'otros', 'neto', 'tipo']
df_aux.columns = cols
df_aux.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28223 entries, 0 to 17661
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   num       28223 non-null  object 
 1   reng      24768 non-null  object 
 2   fecha     28223 non-null  object 
 3   cliente   24284 non-null  object 
 4   vendedor  24284 non-null  float64
 5   almacen   24284 non-null  float64
 6   cantidad  24284 non-null  float64
 7   und       24284 non-null  object 
 8   precio    24284 non-null  float64
 9   base      24284 non-null  float64
 10  iva       24284 non-null  float64
 11  otros     24284 non-null  float64
 12  neto      24284 non-null  float64
 13  tipo      28223 non-null  object 
dtypes: float64(8), object(6)
memory usage: 3.2+ MB


In [13]:
df_aux = df_aux.loc[:,['num', 'fecha', 'cliente', 'vendedor', 'cantidad', 'neto', 'tipo']]
df_aux


,num,fecha,cliente,vendedor,cantidad,neto,tipo
0,00001,PEGA DE CONTACTO/PEGA ZAPATERA (90 ML) ENVASADO,NaN,NaN,NaN,NaN,fa
1,0000006366,2022-01-03 09:00:00,304459.0,1.0,1.0,10.02,fa
2,0000006388,2022-01-07 11:40:00,18419125.0,6.0,1.0,10.34,fa
4,00005,"NIPLE PLASTICO 1"" * 13CMS",NaN,NaN,NaN,NaN,fa
6,0000006409,2022-01-12 10:51:00,13763194.0,7.0,5.0,34.10,fa
...,...,...,...,...,...,...,...
17657,GEN2,GENERICO,NaN,NaN,NaN,NaN,fa
17658,0000003948,2021-04-26 11:19:00,13763881,7.0,1.0,25.43,fa
17659,0000004458,2021-05-15 11:10:00,15862673,7.0,1.0,11.60,fa
17660,0000005244,2021-06-15 10:32:00,15927422,1.0,1.0,4.55,fa


### Construcción del DF definitivo

Los datos en el DF están agrupados por producto: Una línea tiene la información sobre el producto y las siguientes son las facturas que incluyen el producto. 

El objetivo es obtener un DF con la siguiente información por cada línea:

    - num: Número de factura (columna num)
    - fecha: Fecha de la factura (columna fecha)
    - cliente: Código del cliente (columna cliente)
    - vendedor: Código del vendedor (columna vendedor)
    - cod: Código del producto (columna num cuando el largo <=5 )
    - producto: Descripción del producto (columna fecha)
    - cantidad: Cantidad de producto en la factura (columna cantidad)
    - monto: Monto neto del producto en la factura (columna neto)
    - tipo: Tipo de factura (columna tipo)
    
Para eso, recorremos el DF y lo vamos construyendo.

In [14]:
%%time

result = []

cod = ''
prod = ''

for index, row in df_aux.iterrows(): 
    
    if len(row['num']) <= 5 :
        cod = row['num']
        prod = row['fecha']
    else:
        dic = {}
        dic['num'] = row['num']
        dic['fecha'] = row['fecha']
        dic['cliente'] = row['cliente']
        dic['vendedor'] = row['vendedor']
        dic['cod'] = cod
        dic['producto'] = prod
        dic['cantidad'] = row['cantidad']
        dic['monto'] = row['neto']   
        dic['tipo'] = row['tipo'] 
        
        result.append(dic)


CPU times: user 2.99 s, sys: 19.6 ms, total: 3.01 s
Wall time: 3.05 s


In [15]:
df_ventas = pd.DataFrame(result)
df_ventas.sample(30)

,num,fecha,cliente,vendedor,cod,producto,cantidad,monto,tipo
17565,0000001656,2021-02-12 09:33:00,9122473,7.0,01214,BOLSA PLASTICA GRANDE 20 Y 25 kgrs (USO INTERN...,1.0,0.05,fa
14403,0000006223,2021-11-16 10:17:00,17220570,7.0,00391,GRIFERIA LAVAMANOS PLASTICA BLANCA POMO CRUZ G...,15.0,86.25,fa
23729,0000004055,2021-04-29 09:56:00,028133002,11.0,05811,SILICON 290ML CARTUCHO TRANSLUCIDO SIMBI,1.0,30.28,fa
2315,0000000368,2021-07-06 10:09:00,14791866.0,14.0,00515,"REGADOR 1/2"" PLASTICO AMARILLO CON NEGRO/NEGRO...",40.0,319.60,ne
13346,0000000662,2021-01-20 11:09:00,15760949,14.0,00166,RIEGO UNION RAPIDA 32MM AGRO,1.0,1.84,fa
21969,0000002829,2021-03-19 08:13:00,107436754,7.0,04215,"PVC SIFON 75mm/3""CORTO AGUAS NEGRAS AMARILLO",2.0,3.54,fa
9245,0000004948,2021-11-29 15:09:00,9337011.0,7.0,04735,"CAMISA 9"" PELO CORTO CEBRA",1.0,10.00,ne
23797,0000005850,2021-08-11 09:46:00,310060940,10.0,05948,"CEPILLO TRENZADO 4"" * 5/8"" TRUPER CT-610 14187",1.0,82.82,fa
17562,0000001597,2021-02-11 09:17:00,17220588,7.0,01214,BOLSA PLASTICA GRANDE 20 Y 25 kgrs (USO INTERN...,1.0,0.05,fa
1865,0000003820,2021-10-23 08:20:00,5346018.0,7.0,00395,"RIEGO ABRAZADERA PLASTICA 32MM * 3/4"" AGRO/FP",3.0,16.32,ne


In [16]:
df_ventas.describe()

,vendedor,cantidad,monto
count,24284.000000,24284.000000,24284.000000
mean,9.305716,4.522932,49.225811
std,3.513585,22.536477,1638.685625
min,1.000000,0.020000,0.010000
25%,7.000000,1.000000,2.550000
50%,10.000000,1.000000,7.500000
75%,11.000000,3.000000,20.900000
max,14.000000,2000.000000,212610.600000


In [17]:
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24284 entries, 0 to 24283
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   num       24284 non-null  object        
 1   fecha     24284 non-null  datetime64[ns]
 2   cliente   24284 non-null  object        
 3   vendedor  24284 non-null  float64       
 4   cod       24284 non-null  object        
 5   producto  24284 non-null  object        
 6   cantidad  24284 non-null  float64       
 7   monto     24284 non-null  float64       
 8   tipo      24284 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 1.7+ MB


In [32]:
df_ventas['vendedor'] = df_ventas['vendedor'].astype(int)
df_ventas['fecha'] = pd.to_datetime(df_ventas['fecha'])

df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24284 entries, 0 to 24283
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   num       24284 non-null  object        
 1   fecha     24284 non-null  datetime64[ns]
 2   cliente   24284 non-null  object        
 3   vendedor  24284 non-null  int64         
 4   cod       24284 non-null  object        
 5   producto  24284 non-null  object        
 6   cantidad  24284 non-null  float64       
 7   monto     24284 non-null  float64       
 8   tipo      24284 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 1.7+ MB


## Tasa de cambio

Para mitigar los efectos de la inflación, los montos, originalmente en Bolívares (Bs), serán transformados a montos en $USD. Para eso, utilizamos un archivo CSV con todas las tasas de cambio diarias. Luego asignamos la tasa correspondiente a cada archivo según la fecha.

El archivo CSV con las tasas de cambio se crea a partir de un [scraper](https://github.com/dchaconoca/proyecto-ventas/blob/master/scraper_tasa_dolar.ipynb)

In [66]:
FILE_DOLAR = 'datos/in/tasa_dolar.csv'

df_dolar = pd.read_csv(f"{BASE_DIR / FILE_DOLAR}", sep=';')
df_dolar

,fecha,alta,baja
0,02-02-2022,4.69,4.70
1,01-02-2022,4.69,4.74
2,31-01-2022,4.73,4.75
3,30-01-2022,4.73,4.74
4,29-01-2022,4.73,4.75
...,...,...,...
806,19-11-2019,25931.00,33034.00
807,18-11-2019,29027.00,30849.00
808,17-11-2019,28277.00,29509.00
809,16-11-2019,28430.00,29229.00


In [69]:
# Convertimos la columna de fecha
# y extraemos solo los datos a partir del 2021

df_dolar['fecha'] = pd.to_datetime(df_dolar['fecha'], yearfirst=False)
df_dolar = df_dolar.query(' fecha > "2020/12/31" ')
df_dolar

,fecha,alta,baja
0,2022-02-02,4.69,4.70
1,2022-01-02,4.69,4.74
2,2022-01-31,4.73,4.75
3,2022-01-30,4.73,4.74
4,2022-01-29,4.73,4.75
...,...,...,...
393,2021-05-01,1178316.62,1303357.48
394,2021-04-01,1051007.98,1201219.02
395,2021-03-01,1036015.05,1088223.91
396,2021-02-01,1042518.04,1099758.03


In [ ]:
# Los da

## Salvaguarda del DF

In [39]:
%%time
# Guardamos el DF limpio para su análisis posterior
'''
!!! ME DA ERROR 
df_ventas.to_parquet(f"{BASE_DIR / 'datos/out/ventas.parquet'}", 
                    compression='GZIP',
                    engine='pyarrow',
                    object_encoding='utf8')
'''

df_ventas.to_csv(f"{BASE_DIR / 'datos/out/ventas.csv'}", sep=';')


# Guardamos el DF en un archivo Excel para utilizar los datos en Google Data Studio
df_ventas.to_excel(f"{BASE_DIR / 'datos/out/ventas.xlsx'}")


ArrowInvalid: ("Could not convert '18637949' with type str: tried to convert to double", 'Conversion failed for column cliente with type object')

### Archivo Ventas final

In [20]:
# Unimos los 2 archivos de ventas
df_ventas = pd.concat([df_ventas, df_ventas2], axis=0)
df_ventas

NameError: name 'df_ventas2' is not defined

In [ ]:
df_ventas.describe()

In [ ]:
df_ventas_mass1 = df_ventas.query(' Neto > 44 ')
df_ventas_mass1.describe()

In [ ]:
df_ventas_mass2 = df_ventas.query(' Neto > 171 ')
df_ventas_mass2.describe()

In [ ]:
df_ventas_mass3 = df_ventas.query(' Neto > 509 ')
df_ventas_mass3.describe()

In [ ]:
df_ventas_mass4 = df_ventas.query(' Neto > 1346 ')
df_ventas_mass4.describe()

In [ ]:
df_ventas_mass5 = df_ventas.query(' Neto > 2815 ')
df_ventas_mass5.describe()

In [ ]:
df_ventas.plot(x='Fecha', y='Neto')

In [ ]:
max = df_ventas.Neto.max()
df_ventas.query(' Neto == Neto.max() ')

In [ ]:
df_ventas.query(' Fecha > "2022/01/25" ')

In [ ]:
df_ventas['Dia'] = df_ventas.Fecha.dt.weekday
df_ventas['Mes'] = df_ventas.Fecha.dt.month
#df_ventas['Semana'] = df_ventas.Fecha.dt.WeekOfMonth
df_ventas['Año'] = df_ventas.Fecha.dt.year
df_ventas.sample(20)


In [ ]:
ventas_dia_semana = df_ventas.pivot_table('Neto',  index='Dia', aggfunc='count' )
ventas_dia_semana

In [ ]:
ventas_dia_semana.plot.bar()

In [ ]:
ventas_mes_dia = df_ventas.pivot_table('Neto',  index=['Mes', 'Dia', 'Año'], aggfunc='count' )
ventas_mes_dia = pd.DataFrame(ventas_mes_dia)
ventas_mes_dia.reset_index(inplace=True)
ventas_mes_dia

In [ ]:
alt.Chart(ventas_mes_dia).mark_rect().encode(
    x='Dia:O',
    y='Mes:O',
    column='Año:O',
    color='Neto:Q'
)

In [ ]:
ventas_mes_dia.query('Mes == 3')

In [ ]:
%%time

from fastparquet import write

# Guardamos el DF limpio para su análisis posterior
df_ventas.to_parquet(f"{BASE_DIR / 'datos/out/ventas.parquet'}", object_encoding='bytes', compression='gzip')
#write(f"{BASE_DIR / 'datos/out/ventas.parquet'}", df_ventas, compression='GZIP', object_encoding='utf8')
df_ventas.to_excel(f"{BASE_DIR / 'datos/out/ventas.xlsx'}")

### Datos del inventario:

In [ ]:
FILES_STOCK = 'datos/in/2*.xls'
FILES_STOCK

print(f"{BASE_DIR / FILES_STOCK}")

In [ ]:
[Path(filename).name for filename in glob.glob(f"{BASE_DIR / FILES_STOCK}")]

In [ ]:
%%time

df = read_files(FILES_STOCK)

df

In [ ]:
df.describe()

In [ ]:
df.info()

### Limpieza de los datos

- Modificamos el nombre de las columnas para ajustarlos a la información que poseen y facilitar su manejo
- Solo conservaremos: código, nombre, fecha y la información sobre el inventario: Stock inicial, entrdas, ventas y stock final
- Eliminamos las lineas que continen NaN ya que son los totales de los reportes
- Cambiamos el tipo a la columna Código para que sea entero y ocupe menos espacio en memoria
- Transformamos el campo de fecha
- Eliminamos los productos con descripción "0" y null ya que son productos que han sido eliminados del sistema

In [ ]:
# Cambio del nombre de las columnas
df.columns

In [ ]:
cols = ['Codigo', 
        'Unidad',
        'Stock inicial',
        'Costo Stock inic',
        'Compras',
        'CU compras',
        'Compras x CU',
        'Entradas',
        'CU Entradas',
        'Entradas x CU',
        'Ventas',
        'CU Ventas',
        'Ventas x CU',
        'Salida',
        'CU Salida',
        'Salida x CU',
        'Auto consumo',
        'CU auto cons',
        'Salidas/autocons',
        'Costo Stock fin',
        'CU stock fin',
        'Stock final',
        'Producto',
        'Fecha']

In [ ]:
df.columns = cols
df.info()

In [ ]:
# Conservamos solo ciertas columnas

df_stock = df.loc[:,['Fecha', 'Codigo', 'Producto', 'Stock inicial', 'Entradas', 'Ventas', 'Stock final']]
df_stock

In [ ]:
# Buscamos las filas con valores NaN 
df_stock[ df_stock.isnull().values ].sample(40)

In [ ]:
# Eliminamos los registros que contienen NaN ya que corresponden a 
# líneas de totales de los reportes o productos eliminados
df_stock.dropna(inplace=True)

In [ ]:
# Los productos cuya descripción es '0' han sido eliminados del sistema
# Los eliminamos del DF
df_stock.query( "Producto == '0'" )

In [ ]:
df_stock = df_stock.loc[df_stock.Producto != '0' ]


In [ ]:
# Cambiamos el tipo de dato a las columnas Codigo y Fecha
df_stock['Codigo'] = df_stock['Codigo'].astype(int)
df_stock['Fecha'] = pd.to_datetime(df_stock['Fecha'])

df_stock.info()

In [ ]:
df_stock.describe()

In [ ]:
df_stock.sample(20)

## EDA

In [ ]:
df_stock['Mes'] = df_stock.Fecha.dt.month
df_stock['Año'] = df_stock.Fecha.dt.year
df_stock.sample(20)

In [ ]:
ventas_mes_dia = df_stock.pivot_table('Ventas',  index=['Mes', 'Año'], aggfunc='sum' )
ventas_mes_dia = pd.DataFrame(ventas_mes_dia)
ventas_mes_dia.reset_index(inplace=True)
ventas_mes_dia

In [ ]:
alt.Chart(ventas_mes_dia).mark_bar().encode(
    x='Mes:O',
    y='Ventas:O',
    column='Año:O',
    color='Ventas:Q'
)